In [ ]:
import os

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from src.classes.dataset import MRIDataset
from src.classes.shap import ShapExplainer
from src.config import PATH_TO_OUTPUT, PATH_TO_MODELS, PATH_TO_DATASET_CSV, PATH_TO_DATASET, ID_TO_NAME

In [ ]:
# Define required paths
MODEL_PATH = os.path.join(PATH_TO_MODELS, "final-resnet50v.pth")
OUTPUT_PATH = os.path.join(PATH_TO_OUTPUT, "shap_results")

# Load dataset from CSV
df = pd.read_csv(PATH_TO_DATASET_CSV, sep=";", header=0)

# Create a dictionary mapping image indices to (image path, label)
data = {
    idx: (os.path.join(PATH_TO_DATASET, ID_TO_NAME[row["label"]], str(row["img_name"])), row["label"])
    for idx, row in df.iterrows()
}

# Convert labels to numpy array
y = df["label"].to_numpy()

# Generate train-test split
X_indices = np.array(list(data.keys()))
_, test_index = train_test_split(X_indices, test_size=0.2, random_state=42, stratify=y)

# Load dataset using MRIDataset class
dataset = MRIDataset(data)

# Initialize and run SHAP explainer
shap_explainer = ShapExplainer(
    model_path=MODEL_PATH,
    dataset=dataset,
    test_indices=test_index,
    output_path=OUTPUT_PATH
)

shap_explainer.generate_shap_explanations()